In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [30]:
from fastai import *
from fastai.vision import *

In [5]:
path = Path('data/stage2/')
path.ls()

[PosixPath('data/stage2/stage_2_sample_submission.csv'),
 PosixPath('data/stage2/stage_2_train.csv'),
 PosixPath('data/stage2/stage_2_images.zip'),
 PosixPath('data/stage2/images')]

In [39]:
path_img = path/'images'
path_lbl = path/'stage_2_train.csv'

In [40]:
fnames = path_img.ls()
fnames[:3]

[PosixPath('data/stage2/images/ID_d3639ddab.dcm'),
 PosixPath('data/stage2/images/ID_dd3ce7a6d.dcm'),
 PosixPath('data/stage2/images/ID_66ecdb66f.dcm')]

In [7]:
label_df = pd.read_csv(path_lbl)
label_df.head()

,ImageId,EncodedPixels
0,1.2.276.0.7230010.3.1.4.8323329.6904.151787520...,-1
1,1.2.276.0.7230010.3.1.4.8323329.13666.15178752...,557374 2 1015 8 1009 14 1002 20 997 26 990 32 ...
2,1.2.276.0.7230010.3.1.4.8323329.11028.15178752...,-1
3,1.2.276.0.7230010.3.1.4.8323329.10366.15178752...,514175 10 1008 29 994 30 993 32 991 33 990 34 ...
4,1.2.276.0.7230010.3.1.4.8323329.10016.15178752...,592184 33 976 58 956 73 941 88 926 102 917 109...


In [17]:
# Setting div=True in open_mask
class SegmentationLabelList(SegmentationLabelList):
    def open(self, fn): return open_mask(fn, div=True)
    
class SegmentationItemList(SegmentationItemList):
    _label_cls = SegmentationLabelList

# Setting transformations on masks to False on test set
def transform(self, tfms:Optional[Tuple[TfmList,TfmList]]=(None,None), **kwargs):
    if not tfms: tfms=(None,None)
    assert is_listy(tfms) and len(tfms) == 2
    self.train.transform(tfms[0], **kwargs)
    self.valid.transform(tfms[1], **kwargs)
    kwargs['tfm_y'] = False # Test data has no labels
    if self.test: self.test.transform(tfms[1], **kwargs)
    return self
# data_block.ItemLists.transform = transform

In [20]:
SZ = 64

In [37]:
get_y_func = lambda x: 'a'

In [38]:
data = (SegmentationItemList.from_folder(path_img)
        .split_by_rand_pct(0.2, seed=42)
        .label_from_func(get_y_func)
        .databunch())

/home/lgvaz/anaconda3/envs/fastai/lib/python3.7/site-packages/fastai/data_block.py:454: UserWarning: Your training set is empty. If this is by design, pass `ignore_empty=True` to remove this warning.
  warn("Your training set is empty. If this is by design, pass `ignore_empty=True` to remove this warning.")
/home/lgvaz/anaconda3/envs/fastai/lib/python3.7/site-packages/fastai/data_block.py:457: UserWarning: Your validation set is empty. If this is by design, use `split_none()`
                 or pass `ignore_empty=True` when labelling to remove this warning.
  or pass `ignore_empty=True` when labelling to remove this warning.""")


TypeError: object of type 'NoneType' has no len()

In [33]:
data.show_batch()

AttributeError: 'SegmentationItemList' object has no attribute 'show_batch'

In [24]:
# Create databunch
data = (SegmentationItemList.from_folder(path=path_img)
        .split_by_rand_pct(0.2)
        .label_from_func(lambda x : str(x).replace('train', 'masks'), classes=[0, 1])
#         .add_test((path/'test').ls(), label=None)
        .transform(get_transforms(), size=SZ, tfm_y=True)
        .databunch(bs=32)
        .normalize(imagenet_stats))

In [28]:
data.

2

In [41]:
def open_dcm_image(fn:PathOrStr, div:bool=True, convert_mode:str='RGB', cls:type=Image,
        after_open:Callable=None)->Image:
    "Return `Image` object created from image in file `fn`."
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning) # EXIF warning from TiffPlugin
        #x= PIL.Image.open(fn).convert(convert_mode)
        # code added for opening dcm images
        dicom_file = pydicom.dcmread(str(fn))
        arr = dicom_file.pixel_array.copy() 
        arr = arr * int(dicom_file.RescaleSlope) + int(dicom_file.RescaleIntercept) 
        level = 40; window = 80
        arr = np.clip(arr, level - window // 2, level + window // 2)
        x = Image.fromarray(arr).convert(convert_mode)
    if after_open: x = after_open(x)
    x = pil2tensor(x,np.float32)
    if div: x.div_(255)
    return cls(x)

In [43]:
open_dcm_image(fnames[0])

NameError: name 'pydicom' is not defined